In [2]:
import src.utils as u
import src.model as m

In [3]:
print(u.return1())

AttributeError: module 'src.utils' has no attribute 'return1'

In [2]:
resolution = 224
dataset = u.data_loader("./dataset/dataset/", resolution, resolution)
labels = u.get_labels(dataset)
(training, validation, testing) = u.stratified_distribution(dataset)

ValueError: Found input variables with inconsistent numbers of samples: [19182, 21314]

In [8]:
epochs = 10
model = m.VisionModel(len(labels), resolution)

m.train_model(model, epochs, training, validation)
m.evaluate(model, testing)

/opt/conda/lib/python3.11/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 1/10
Train loss: 3.5571
Validation loss: 2.9542, Validation Accuracy: 48.86
Epoch: 2/10
Train loss: 2.7924
Validation loss: 2.8421, Validation Accuracy: 47.92


KeyboardInterrupt: 